In [1]:
%load_ext autoreload
%autoreload
%matplotlib inline

from news_crawler import NewsCrawler
from IPython.display import display
from IPython.core.debugger import set_trace

# 0. 초기화

In [6]:
crawler = NewsCrawler(storage='local')

checking local storage... done
we have total 297989 articles (285232 downloaded, 12757 trashed)


# 1. URL을 모은다

In [7]:
# 제거고려: nextbigfuture, nbcnews, joongang, chosun

In [8]:
collect_result = crawler.collect(); display(*collect_result)

100.00% (321.18 seconds): Collecting URLs... usatoday            

100.00% (0.40 seconds): Selecting URLs... cryptonews          

,collected,collected_unique,selected
all,25611,25542,1436
reuters,1114,1114,183
euronews,1088,1088,146
yahoo,378,378,106
mirror,180,180,96
dailymail,2756,2756,93
businessinsider,577,577,87
investing.com,881,881,71
usatoday,515,515,54
rt,440,440,50


,url,pubs,actual_pub
0,https://aljazeera.com/news/2019/10/dozen-kille...,"aljazeera, axios",aljazeera
1,https://aljazeera.com/news/2019/10/lebanon-pm-...,"aljazeera, axios",aljazeera
2,https://bbc.com/news/uk-50221856,"bbc, realclearpolitics",bbc
3,https://businessinsider.co.za/boeing-ceo-senat...,"businessinsider, news24",businessinsider
4,https://businessinsider.co.za/boris-johnson-cl...,"businessinsider, news24",businessinsider
5,https://businessinsider.co.za/fnb-affordable-h...,"businessinsider, news24",businessinsider
6,https://foxnews.com/world/how-al-baghdadi-isis...,"fox, realclearpolitics",fox
7,https://hotair.com/archives/ed-morrissey/2019/...,"townhall, hotair",hotair
8,https://msnbc.com/msnbc/watch/as-trump-takes-a...,"msnbc, nbcnews",msnbc
9,https://nbcnews.com/better/lifestyle/owning-do...,"msnbc, nbcnews",nbcnews


# 2. 해당 URL의 뉴스를 다운로드한다

In [9]:
download_result = crawler.download(); download_result

100.00% (1164.02 seconds): Downloading... euronews            

collect                             download          \
                     collected collected_unique selected downloaded trashed   
all                      25611            25542     1436        847     589   
reuters                   1114             1114      183         85      98   
euronews                  1088             1088      146          8     138   
yahoo                      378              378      106         75      31   
mirror                     180              180       96         88       8   
dailymail                 2756             2756       93         77      16   
businessinsider            577              577       87         70      17   
investing.com              881              881       71         31      40   
usatoday                   515              515       54         15      39   
rt                         440              440       50         11      39   
independent                723              723       32         31       1   
breitbart                  391              391       31         29       2   
telegraph                  820              820       30         25       5   
marketwatch                189              189       29         21       8   
metro                      250              250       29         27       2   
fox                        345              345       27         17      10   
cnn                        835              835       25         12      13   
aljazeera                  479              479       24          3      21   
news24                    1202             1199       22         17       5   
axios                      134              129       21         16       5   
financialpost              112              112       17         17       0   
chinadaily                 690              690       17          4      13   
cnbc                       177              177       15         11       4   
washingtonpost             129              129       14         13       1   
blastingnews               124              124       14          0      14   
washingtontimes            459              459       14         14       0   
guardian                   157              157       12         11       1   
thestreet                  130              130       12          5       7   
wsj                        103              103       10          9       1   
abcnews                     99               99       10         10       0   
...                        ...              ...      ...        ...     ...   
thenextweb                  81               81        5          5       0   
msnbc                      136              100        4          1       3   
vice                       104              104        4          4       0   
nytimes                     99               99        4          4       0   
chosun                     286              286        4          0       4   
techcrunch                  20               20        3          3       0   
cryptonews                 328              328        3          1       2   
npr                        919              919        3          3       0   
realclearpolitics          154              135        3          0       3   
indiatimes                 171              171        3          3       0   
atlantic                    99               99        2          2       0   
dailybeast                  50               50        2          0       2   
politico                   205              205        2          0       2   
japan-news                  71               71        2          0       2   
cointelegraph               71               71        1          1       0   
newrepublic                 20               20        1          1       0   
ccn                          0                0        0          0       0   
joongang                   368              368        0          0      

In [28]:
download_result.to_dict()

{('collect', 'collected'): {'all': 8645,
  'reuters': 764,
  'fox': 299,
  'investing.com': 510,
  'cnn': 623,
  'telegraph': 542,
  'news24': 270,
  'marketwatch': 137,
  'washingtonpost': 137,
  'wsj': 204,
  'aljazeera': 168,
  'thestreet': 128,
  'washingtontimes': 115,
  'nationalreview': 730,
  'guardian': 164,
  'financialpost': 113,
  'msnbc': 136,
  'nbcnews': 87,
  'zdnet': 239,
  'businessinsider': 76,
  'cnbc': 66,
  'bbc': 192,
  'afr': 86,
  'vice': 105,
  'techcrunch': 20,
  'axios': 21,
  'politico': 206,
  'realclearpolitics': 139,
  'nytimes': 99,
  'theverge': 947,
  'scmp': 252,
  'thehill': 88,
  'huffpost': 65,
  'chinadaily': 393,
  'morningstar': 32,
  'rt': 16,
  'indiatimes': 33,
  'cryptonews': 327,
  'time': 19,
  'japan-news': 70,
  'townhall': 0,
  'arirang': 0,
  'yahoo': 16,
  'americanconservative': 0,
  'atlantic': 0,
  'arynews': 0,
  'usatoday': 0,
  'thenextweb': 0,
  'nymag': 0,
  'thinkprogress': 0,
  'arstechnica': 0,
  'newrepublic': 0,
  'npr':

In [13]:
10000/50

200.0

Bigquery와 local 뉴스 갯수가 다를 것이다. Bigquery에 뉴스를 모두 업데이트 한 후 해당 repository에 문제가 생겨서, github에서 전일기준 데이터를 clone하여, 뉴스를 다시 다운로드 받은 날이 있다. 2019-10-22

In [15]:
275632+81

275713

In [18]:
from pathlib import Path
downloaded = [p.stem for p in Path('newsdata/downloaded').glob('**/*.json')]
trashed = [p.stem for p in Path('newsdata/trashed').glob('**/*.json')]

In [19]:
fnames_remove = set(downloaded) & set(trashed)

In [20]:
fnames_remove

{'364359fe8009b0eafc8c7698d136c85609507832',
 '48c17b8c7052dd982fec7893ad220f309311523e',
 '491d72f7e9c206d80afec8be4af247d2cee62b2d',
 '521cd5b023984b3dcd764aaf26e56a82e4bdf864',
 '6b3253676b0ac3b421e3b75138acf378edb867d4',
 '6d2fd1f6e66da4710074216d15f1f96fb1894355',
 'a02a1f32ccae0a118c17506ffad63bd7a31b84f3',
 'c82ed757f10fdfd53c28cb364e63bccf9947bef3',
 'f86363446716010bbad6138fe5ed767a36250441',
 'fce755ae86014651c8c3bf7061a7a0a6ca348460'}

In [21]:
for fname in fnames_remove:
    file = Path('newsdata/trashed/' + fname[:3] + '/' + fname + '.json')
    file.unlink()